<a href="https://colab.research.google.com/github/1900690/Water-meter-reading/blob/main/watermeter_analog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 必要なライブラリのインストール
!pip install gradio

In [2]:
import cv2
import math
import numpy as np
import gradio as gr

# --- 設定（定数） ---
RED_LOWER_1 = np.array([0, 100, 100])
RED_UPPER_1 = np.array([10, 255, 255])
RED_LOWER_2 = np.array([160, 100, 100])
RED_UPPER_2 = np.array([180, 255, 255])

SCALE_UNIT = 36.0  # 36度ごとに1ユニット
COLOR_CYAN = (0, 255, 255)  # 水色 (RGB)
COLOR_GREEN = (0, 255, 0)   # ガイド線
COLOR_RED = (255, 0, 0)     # 中心点

def get_needle_mask(hsv_img):
    """HSV画像から赤色のマスクを作成する"""
    mask1 = cv2.inRange(hsv_img, RED_LOWER_1, RED_UPPER_1)
    mask2 = cv2.inRange(hsv_img, RED_LOWER_2, RED_UPPER_2)
    return cv2.bitwise_or(mask1, mask2)

def calculate_meter_value(tip_pos, center_pos):
    """中心と先端の座標から角度とメーター値を計算する"""
    tx, ty = tip_pos
    cx, cy = center_pos

    dx = tx - cx
    dy = ty - cy

    # 真上(0, -1)を0度とした時計回りの角度
    angle_rad = math.atan2(dx, -dy)
    angle_deg = np.degrees(angle_rad) % 360

    return angle_deg / SCALE_UNIT

def analog_meter_app(input_img):
    if input_img is None:
        return None

    # 1. 準備
    h, w = input_img.shape[:2]
    cx, cy = w // 2, h // 2
    res_img = input_img.copy()

    # 2. 赤色抽出
    # GradioはRGBで渡すため、色空間の変換に注意
    bgr_img = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    full_mask = get_needle_mask(hsv)

    # 3. 針の先端特定
    contours, _ = cv2.findContours(full_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    tip_x, tip_y = cx, cy  # デフォルト値
    meter_value = 0.0

    if contours:
        # 最大の領域（針）を特定
        cnt = max(contours, key=cv2.contourArea)

        # 中心から最も遠い点を先端とする
        dists = np.sum((cnt.reshape(-1, 2) - [cx, cy])**2, axis=1)
        max_idx = np.argmax(dists)
        tip_x, tip_y = cnt.reshape(-1, 2)[max_idx]

        # 4. 数値計算
        meter_value = calculate_meter_value((tip_x, tip_y), (cx, cy))

    # 5. 描画
    # ガイド線と中心点
    cv2.line(res_img, (cx, cy), (tip_x, tip_y), COLOR_GREEN, 2)
    cv2.circle(res_img, (cx, cy), 3, COLOR_RED, -1)

    # 数値（水色）
    display_text = f"{meter_value:.1f}"
    font_scale = w / 250
    cv2.putText(res_img, display_text, (15, h - 20),
                cv2.FONT_HERSHEY_SIMPLEX, font_scale, COLOR_CYAN, 2, cv2.LINE_AA)

    return res_img

# --- UI定義 ---
app = gr.Interface(
    fn=analog_meter_app,
    inputs=gr.Image(type="numpy", label="メーター画像をアップロード"),
    outputs=gr.Image(type="numpy", label="解析結果"),
    title="Analog Meter Reader v2",
    description="赤色の針を検出し、真上を0とした36度間隔の数値を算出します。",
    allow_flagging="never"
)

if __name__ == "__main__":
    app.launch(inline=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9c17fab897dd4a7fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
